In [ ]:
"""
1 YOLO local image'lerden çalıştırılacak
2 YOLO local videodan opencv ile frame okuyarak çalıştırılacak
3 YOLO local videodan gstreamer ile frame okuyarak çalıştırılacak

4 RGB2Depth modelleri denenecek en iyisi bulunacak
5 RGB2D modeli local image'lerden çalıştırılacak
6 RGB2D modeli local videodan gstreamer ile frame okuyarak çalıştırılacak

7 YOLO ve RGB2D modeli birlikte çalıştırılacak
8 modellerin çıktıları rapordaki gibi formatlanacak

9 örnek çıktılarla prompt denemeleri yapılacak 
10 LLM olarak gruba attığım python librarysi incelenecek
11 LLMden alınan çıktı text2speech olarak ayarlanacak
"""

In [ ]:
"""
cv.namedWindow('Screen Image', cv.WINDOW_NORMAL)

cv.imshow('Screen Image', screen)
cv.waitKey(0)
cv.destroyAllWindows()
"""

## Model

In [1]:
import torch
import os
import numpy as np
import cv2 as cv
import time


model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/meric/.cache/torch/hub/ultralytics_yolov5_master
requirements: Ultralytics requirement "gitpython>=3.1.30" not found, attempting AutoUpdate...
requirements: ❌ AutoUpdate skipped (offline)
YOLOv5 🚀 2023-6-14 Python-3.9.13 torch-2.0.0+cu117 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


## Local Images

In [10]:
def put_bboxs(img, result, threshold):
    object_ids = []
    id_counter = 1
    for bbox, label, conf in zip(result.xyxy[0][:,:4], result.pred[0][:,5], result.pred[0][:,4]):
        if conf >= threshold:
            x1, y1, x2, y2 = map(int, bbox[:4])
            class_name = result.names[int(label)]
            
            cv.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 7)
            
            label_text = str(id_counter) + "-" + str(class_name) + ": {:.2f}".format(conf)
            cv.putText(img, label_text, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 0), 6)
            
            object_ids.append(id_counter)
            id_counter = id_counter + 1
        else:
            # -1 id means model is not confident enough
            object_ids.append(-1)

    return object_ids

In [11]:
def calculate_angles(img, result):
    # list of objects center pixels offsets according to images center point
    # negative: left/up
    # positive: right/down
    # X:row axis, determines how high or low an object is
    # Y:column axis, determines how left or right an object is
    im_s = img.shape

    # finding the angles of object according to camera, most left(y=-1):-50 degrees, most right(y=+1):+50 degrees
    object_angles = [(
                    round((((c[1] + c[3])/2 - im_s[0]/2) / (im_s[0]/2) * 5).item())*10,
                    round((((c[0] + c[2])/2 - im_s[1]/2) / (im_s[1]/2) * 5).item())*10)
                    for c in result.xyxy[0][:,:4]]

    return object_angles

In [12]:
def display_img(img):
    cv.namedWindow('result', cv.WINDOW_NORMAL)
    cv.imshow('result', img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
folder_path = "./YOLO_test_images"
images = os.listdir(folder_path)

for img_name in images:
    img = cv.imread(os.path.join(folder_path, img_name))
    
    result = model(img)

    object_ids = put_bboxs(img, result, 0.60)
    object_angles = calculate_angles(img, result)
    object_labels = [result.names[int(l)] for l in result.pred[0][:,5]]
    
    display_img(img)

In [14]:
print("ID\tCLASS\tLEFT-RIGHT ANGLE")
for id, l, d in zip(object_ids, object_labels, object_angles):
    print(id,"\t",l,"   ","\t",d)

ID	CLASS	LEFT-RIGHT ANGLE
1 	 person     	 (30, -20)
2 	 person     	 (30, -20)
3 	 bench     	 (30, 20)
4 	 person     	 (30, -20)
5 	 person     	 (20, 0)
6 	 person     	 (30, 40)
7 	 person     	 (20, 0)
8 	 person     	 (20, -10)
9 	 person     	 (30, -10)
10 	 person     	 (20, -10)
11 	 person     	 (30, 10)
12 	 person     	 (30, 20)
13 	 person     	 (20, 10)
14 	 person     	 (20, 0)
-1 	 person     	 (20, 0)


## Local Video

In [ ]:
video_path = './YOLO_test_videos/video1.webm'
cap = cv.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

frame_rate = 5
frame_interval = 1 / frame_rate

while True:
    start_time = time.time()
    ret, img = cap.read()
    if not ret:
        break

    result = model(img)

    object_ids = put_bboxs(img, result, 0.60)
    object_angles = calculate_angles(img, result)
    object_labels = [result.names[int(l)] for l in result.pred[0][:,5]]
    
    cv.namedWindow('result', cv.WINDOW_NORMAL)
    cv.imshow('result', img)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    elapsed_time = time.time() - start_time
    time_to_wait = max(0, frame_interval - elapsed_time)
    time.sleep(time_to_wait)

cap.release()
cv.destroyAllWindows()

## Gstreamer

In [19]:
video_path = './YOLO_test_videos/video1.webm'
pipeline = f'filesrc location={video_path} ! matroskademux ! vp8dec ! videoconvert ! appsink'

cap = cv.VideoCapture(pipeline, cv.CAP_GSTREAMER)

if not cap.isOpened():
    print("Error: Could not open video file.")
    print(8/0)

frame_rate = 5
frame_interval = 1 / frame_rate

while True:
    start_time = time.time()
    ret, img = cap.read()
    if not ret:
        break

    result = model(img)

    object_ids = put_bboxs(img, result, 0.60)
    object_angles = calculate_angles(img, result)
    object_labels = [result.names[int(l)] for l in result.pred[0][:,5]]
    
    cv.namedWindow('result', cv.WINDOW_NORMAL)
    cv.imshow('result', img)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    elapsed_time = time.time() - start_time
    time_to_wait = max(0, frame_interval - elapsed_time)
    time.sleep(time_to_wait)

cap.release()
cv.destroyAllWindows()


Error: Could not open video file.


ZeroDivisionError: division by zero

In [21]:
cap.isOpened()

False